In [1]:
import os

import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

import cmcrameri.cm as cmc
import math

In [2]:
def myround(x, base=1):
    return base * round(x/base)

In [3]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined.csv')

In [4]:
# looks for time value of root
def f3(end_time, total_moment, time_opt, momentrate_opt, start, points_before_zero, proportion = 0.1):

    dx = time_opt[1]-time_opt[0]
    end_window = (end_time/dx)+points_before_zero
    end = int(np.floor(end_window))
    #print(start, end)
    if start == end:
        end += 1
    short = scipy.integrate.simpson(momentrate_opt[start:end], dx = dx)
    return short-(total_moment*proportion)

In [5]:
proportions_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

all_durations = []
all_root_times = []
all_moments = []
all_relative_root_times = []

results_db = pd.DataFrame(columns = ['scardec_name', 'magnitude', 'int_magnitude', 'year', 'month', 'day', '10', '20', '30', '40', '50', '60', '70', '80', '90'])

colors = {'10': 'dodgerblue',
          '20': 'darkorange',
          '30': 'green',
          '40': 'red',
          '50': 'purple',
          '60': 'blue',
          '70': 'orange',
          '80': 'green',
          '90': 'red'}


In [6]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined_scardec_ye_usgs_sigloch_isc_mag.csv')

In [7]:
combined
combined.columns = ['events', 'scardec', 'ye', 'isc', 'sigloch', 'usgs', 'mag']

In [8]:
catalog = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined_m55_catalog.csv', sep = '|')

In [9]:
cols = catalog.columns
column_names = []
for c in cols:
    column_names.append(c.strip().rstrip().lower())
column_names[0] = 'catalog_id'
catalog.columns = column_names

In [10]:
catalog['year'] = catalog.apply(lambda x: x['time'][:4], axis = 1)
catalog['month'] = catalog.apply(lambda x: x['time'][5:7], axis = 1)
catalog['day'] = catalog.apply(lambda x: x['time'][8:10], axis = 1)
catalog['hour'] = catalog.apply(lambda x: x['time'][11:13], axis = 1)
catalog['minute'] = catalog.apply(lambda x: x['time'][14:16], axis = 1)

catalog['event'] = catalog.apply(lambda x: x['year'] + x['month'] + x['day'] + '_' + x['hour'] + x['minute'], axis = 1)

catalog['int_magnitude'] = catalog.apply(lambda x: myround(x['magnitude']), axis = 1)

In [11]:
catalog.drop(columns = ['contributor', 'contributorid', 'magauthor', 'eventlocationname', 'author', 'catalog', 'time'], inplace = True)

In [12]:
catalog = catalog[['event', 'catalog_id', 'year', 'month', 'day', 'hour', 'minute', 'latitude', 'longitude', 'depth/km', 'magnitude', 'int_magnitude', 'magtype']]

In [19]:
results_db

,event,10,20,30,40,50,60,70,80,90
0,19920120_1337,0.323944,0.394366,0.436620,0.450704,0.507042,0.563380,0.633803,0.718310,0.788732
1,19920213_0129,0.348485,0.469697,0.611111,0.686869,0.747475,0.792929,0.833333,0.863636,0.909091
2,19920305_1439,0.330000,0.420000,0.480000,0.520000,0.560000,0.610000,0.650000,0.690000,0.790000
3,19920307_0153,0.309524,0.373016,0.428571,0.468254,0.507937,0.539683,0.603175,0.666667,0.761905
4,19920313_1601,0.295238,0.342857,0.380952,0.409524,0.447619,0.485714,0.571429,0.647619,0.771429
...,...,...,...,...,...,...,...,...,...,...
4078,20221208_0050,0.547619,0.595238,0.642857,0.690476,0.738095,0.738095,0.785714,0.833333,0.880952
4079,20221211_1431,0.351064,0.436170,0.500000,0.531915,0.585106,0.617021,0.680851,0.734043,0.829787
4080,20221214_1840,0.383333,0.433333,0.466667,0.550000,0.583333,0.650000,0.700000,0.766667,0.850000
4081,20221220_1034,0.423810,0.519048,0.638095,0.690476,0.723810,0.757143,0.800000,0.852381,0.928571


In [23]:
for dataset in ['scardec', 'ye', 'usgs', 'isc', 'sigloch']:
	results_db = pd.read_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/{dataset}_relative_root_times.csv')
	results_db.columns = ['event', '10', '20', '30', '40', '50', '60', '70', '80', '90']
	results_db.sort_values(by = 'event', inplace = True, ignore_index=True)

	results_10 = pd.DataFrame(columns = ['event', '0_10', '10_20', '20_30', '30_40', '40_50', '50_60', '60_70', '70_80', '80_90', '90_100'])

	if dataset == 'sigloch':
		results_10['event'] = results_db.apply(lambda x: x['event'][:-2], axis = 1)
	else:
		results_10['event'] = results_db['event']
	
	results_10['0_10'] = results_db['10']
	results_10['10_20'] = results_db['20'] - results_db['10']
	results_10['20_30'] = results_db['30'] - results_db['20']
	results_10['30_40'] = results_db['40'] - results_db['30']
	results_10['40_50'] = results_db['50'] - results_db['40']
	results_10['50_60'] = results_db['60'] - results_db['50']
	results_10['60_70'] = results_db['70'] - results_db['60']
	results_10['70_80'] = results_db['80'] - results_db['70']
	results_10['80_90'] = results_db['90'] - results_db['80']
	results_10['90_100'] = 1 - results_db['90']

	results_10_and_catalog = pd.merge(results_10, catalog, on = 'event', how = 'inner')
	results_10_and_catalog = results_10_and_catalog[['event', 'catalog_id', 'year', 'month', 'day', 'hour', 'minute', 'latitude', 'longitude', 'depth/km', 'magnitude', 'int_magnitude', 'magtype', '0_10', '10_20', '20_30', '30_40', '40_50', '50_60', '60_70', '70_80', '80_90', '90_100']]

	results_10_and_catalog.to_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/{dataset}_results_10_all.csv', index = False)

	if dataset == 'sigloch':
		results_10_and_catalog_filtered = results_10_and_catalog[results_10_and_catalog['magnitude'] <= 7.2].copy()
	else:
		results_10_and_catalog_filtered = results_10_and_catalog.copy()

	results_10_and_catalog_filtered.to_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/{dataset}_results_10.csv', index = False)

